In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import idaes
import scipy.stats as stats
from fim_doe import *

[    0.00] Initializing mpi-sppy


In [2]:
from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

# Import configuration
#from hfc32_emimtf2n_PR import configuration 

In [3]:
from hfc32_emimtf2n_PR import configuration 

In [4]:
from generalize_functions import PRModels

### import models

In [5]:
# Model design variables 
data_exp = pd.read_csv('./emimtf2n/R32/Final_Results/MBDoE/r32_emimtf2n_subset.csv')

data_exp.head()

,temperature,pressure,x_R32,x_emimTf2N
0,283.15,100100,0.136,0.864
1,283.15,250300,0.306,0.694
2,283.15,399300,0.448,0.552
3,283.15,549300,0.570,0.430
4,283.15,700400,0.672,0.328


In [6]:
print(data_exp.iloc[1]['x_R32'])

0.306


In [7]:
# Model params
params = pd.read_csv('./emimtf2n/R32/Final_Results/MBDoE/Params/PR_params_3params_Opt1.csv',header=None)

# 3 parameters, kappa A [comp1, comp2], kappa A [comp2, comp1], kappaB [comp1, comp2]
params_list = list(params[0])
print(params_list)

[0.05611526243688785, -0.11697708014458684, -0.09822244994811687, 0.0, 0.0, 0.0, 0.0, 0.0]


In [9]:

model_creation = PRModels(params_list, configuration, 
                         comp_1= "R32", comp_2 = "emimTf2N", 
                         x_comp_1="x_R32", x_comp_2="x_emimTf2N")

print(model_creation.PR_kappa_A_comp_1_comp_2)

mod = model_creation.create_model(data_exp.iloc[1])

0.05611526243688785
2022-10-11 11:52:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.


## Use Pyomo.DOE

In [10]:
# Create model function
# change the data row by changing i here;


createmod = mod

# Control time set [h]
t_control = [0]

# design variable and its control time set
# (What goes here does not matter because we tell Pyomo.DOE that it does not need to fix design variables)
dv_pass = {'fs.F101.inlet.temperature': t_control,
           'fs.F101.inlet.pressure': t_control,
         'fs.F101.inlet.mole_frac_comp[0,"R32"]': t_control,
          "fs.F101.inlet.mole_frac_comp[0,'emimTf2N']":t_control}
    
# Create measurement object
#measure_pass = {'fs.F101.control_volume.properties_out[0.0].pressure': t_control}
measure_pass = {'fs.state_block.pressure': t_control}
measure_class =  Measurements(measure_pass)


# prior information: none
prior_pass = [[0]]


All measurements are flattened.
Flatten measurement name: ['fs.state_block.pressure']


In [11]:
# Define parameter nominal value 
parameter_dict = { 'fs.properties.PR_kappa_A["R32", "emimTf2N"]': params[0][0], 
                    'fs.properties.PR_kappa_A["emimTf2N", "R32"]': params[0][1],
                  'fs.properties.PR_kappa_B["R32","emimTf2N"]': params[0][2]}



In [12]:
sensi_opt = 'direct_kaug'
    
# Define experiments and design variable names 
# This does not matter for this problem but needed for inputs
exp1 = {'fs.F101.inlet.temperature': {0: 1},
           'fs.F101.inlet.pressure': {0: 1},
         'fs.F101.inlet.mole_frac_comp[0,"R32"]': {0: 1},
          "fs.F101.inlet.mole_frac_comp[0,'emimTf2N']": {0: 1}}

design_names = ['model.fs.F101.inlet.temperature[0]', 'model.fs.F101.inlet.pressure[0]',
                'model.fs.F101.inlet.mole_frac_comp[0,"R32"]', 'fs.F101.inlet.mole_frac_comp[0,"emimTf2N"]']

In [13]:

# create object
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass,fixed=True)


# compute FIM for a square MBDOE problem
# Note that I did not scale the Jacobian 
result = doe_object.compute_FIM(exp1,mode=sensi_opt, FIM_store_name = 'dynamic.csv', 
                                scale_nominal_param_value=True, 
                                store_output = 'store_output', read_output=None,
                                formula='central')

# calculate FIM 
result.calculate_FIM(doe_object.design_values)


Sensitivity information is scaled by its corresponding parameter nominal value.
After practice: {'fs.state_block.pressure': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from

### Information from k_aug

You could print out "raw" information from k_aug (not necessarily). You can find the Jacobian information for the measurement (pressure) in one line.

In [ ]:
for _ in range(len(doe_object.dsdp_col)):
    print('Variable component:', doe_object.dsdp_col[_])
    print('Gradients:', doe_object.dsdp_array[_])

### Jacobian matrix and FIM

In [14]:
print(result.jaco_information)
print(result.FIM)

{'fs.properties.PR_kappa_A["R32", "emimTf2N"]': [124240.27056312615], 'fs.properties.PR_kappa_A["emimTf2N", "R32"]': [-12718.26189439893], 'fs.properties.PR_kappa_B["R32","emimTf2N"]': [206856.48155151054]}
[[ 1.54356448e+10 -1.58012030e+09  2.56999052e+10]
 [-1.58012030e+09  1.61754186e+08 -2.63085491e+09]
 [ 2.56999052e+10 -2.63085491e+09  4.27896040e+10]]


In [15]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

======Result summary======
Four design criteria log10() value:
A-optimality: 10.766316183338489
D-optimality: nan
E-optimality: nan
Modified E-optimality: nan


/home/parallels/anaconda3/envs/pyomo/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log10
  after removing the cwd from sys.path.
/home/parallels/anaconda3/envs/pyomo/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log10
  """
/home/parallels/anaconda3/envs/pyomo/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log10
  
